# Exploración de modelos de clasificación en la recuperación de información
## Objetivo
Comprender la aplicación práctica y las diferencias entre el modelo de espacio vectorial y el modelo de independencia binaria en la clasificación de documentos en relación con una consulta de usuario.

## Paso 1: Preprocesamiento de datos
Asegúrese de que los documentos siguen cargados y preprocesados desde la tarea anterior. Los datos deben estar limpios y listos para una consulta avanzada. Escriba una función para cargar y preprocesar los documentos de texto desde un directorio especificado. Este paso implica leer cada archivo, convertir el texto a minúsculas para un procesamiento uniforme y almacenar los resultados en un diccionario.

In [1]:
import os
from pathlib import Path
import re
#from collections import defaultdict
from collections import Counter
import numpy as np

In [2]:
def buscar_titulos(directorio):
    archivos_titulos = []
    # Iterar sobre todos los archivos en el directorio
    for archivo in directorio.glob('*.txt'):
            # Abrir el archivo y buscar la palabra
            titulo=archivo.stem
            archivos_titulos.append(str(titulo))
    return archivos_titulos

In [3]:
def diccionario_libros(directorio):
    palabras_individuales = []
    for archivo in directorio.glob('*.txt'):
            with open((archivo), 'r', encoding='utf-8') as file:   
             contenido = file.read()
            # Limpiar el contenido: eliminar caracteres especiales y dividir en palabras
             palabras = re.findall(r'\b[A-Za-z0-9]+\b', contenido.lower())
             palabras_individuales.extend(palabras)      
    return palabras_individuales

In [4]:
directorio_a_buscar = Path("data")  # Ruta de la carpeta "data"
titulos_encontrados = buscar_titulos(directorio_a_buscar)


In [5]:
palabras_diccionario=diccionario_libros(directorio_a_buscar)
palabras_diccionario= set(palabras_diccionario)
palabras_ordenadas = sorted(palabras_diccionario)

In [6]:
filas=len(palabras_diccionario)
columnas=len(titulos_encontrados)
#matrizZeros = np.zeros((filas, columnas), dtype=int)
#print(matrizZeros)

## Paso 2: Modelo vectorial espacial (VSM)
### Tarea: Implementar un Modelo de Espacio Vectorial simple utilizando la frecuencia de términos.

### Requisitos:

Representación de documentos y consultas: Convertir cada documento y la consulta en un vector donde cada dimensión corresponda a un término del corpus. Utilizar la frecuencia de términos simple para la ponderación.
Cálculo de la similitud coseno: Calcular la similitud coseno entre el vector de consulta y cada vector de documento.

In [10]:
import numpy as np

# Cargar la matriz desde el archivo "matriz_guardada.npy"
matriz_cargada = np.load("matriz_de_Conteo.npy")

# Ahora puedes usar la matriz cargada en tu código
print(matriz_cargada)

[[ 7  0  1 ...  0  0  0]
 [ 1  0  0 ...  0  0  0]
 [23  1  1 ...  9  1  1]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


In [12]:
palabra_a_buscar = input("Ingresa la palabra que quieres buscar: ") 

Ingresa la palabra que quieres buscar: that hola como estas


In [13]:
def palabras_input(palabra_input):
    palabras_ingresadas = []
    palabras = re.findall(r'\b[A-Za-z0-9]+\b',palabra_input.lower())
    palabras_ingresadas.extend(palabras) 
    return palabras_ingresadas

In [14]:
palabra_dict = palabras_input(palabra_a_buscar)
print(palabra_dict)

['that', 'hola', 'como', 'estas']


In [15]:
matrizZeros2 = np.zeros((filas, columnas), dtype=int)
print(matrizZeros2)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [16]:
def palabras_por_libro4(directorio, libro_nombre, input_palabra):
    palabras_input = []
    archivo = directorio / (libro_nombre + '.txt')
    # Verificar si el archivo existe
    if archivo.exists():
        with open(archivo, 'r', encoding='utf-8') as file:
            contenido = file.read()
            # Limpiar el contenido: eliminar caracteres especiales y dividir en palabras
            palabras_individuales = re.findall(r'\b[A-Za-z0-9]+\b', contenido.lower())
            for palabra in input_palabra:
                if palabra.lower() in palabras_individuales:  # Verificar si la palabra completa está presente
                    palabras_input.append(palabra)
    return palabras_input


In [18]:
for x, titulo2 in enumerate(titulos_encontrados):
   # contador_palabras=1
   # print(matriz[j])
    palabras_por_libro_actual4= palabras_por_libro4(directorio_a_buscar,titulo2,palabra_dict)
   # print(palabras_por_libro_actual4)
    for y, palabra2 in enumerate(palabras_ordenadas):
        if palabra2 in palabras_por_libro_actual4:
            matrizZeros2[y, x] = 1

In [19]:
print(matrizZeros2)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [26]:
#162907, 184120

In [27]:
def multiplicar_matrices(matriz1, matriz2):
    matriz1_np = np.array(matriz1)
    matriz2_np = np.array(matriz2)
    resultado = np.multiply(matriz1_np, matriz2_np)
    #return resultado.tolist()
    return resultado

In [28]:
resultado_matrices= multiplicar_matrices(matriz_cargada, matrizZeros2)

## Paso 3: Modelo binario de independencia (BIM)
### Tarea: Implementar un Modelo de Independencia Binaria básico para clasificar documentos.

### Requisitos:

Representación binaria: Representar el corpus y la consulta en vectores binarios (1 si el término está presente, 0 en caso contrario).
Estimación de probabilidades: Asumir probabilidades arbitrarias para la presencia de cada término en documentos relevantes y no relevantes.
Puntuación de relevancia: Calcular la puntuación de relevancia de cada documento a partir del producto de las probabilidades de los términos presentes en la consulta.

In [30]:
resultado_final=[]
for l, titulo2 in enumerate(titulos_encontrados):
    fila1 = matriz_cargada[:,l]
    modulo1 = np.linalg.norm(fila1)
    fila2 = matrizZeros2[:,l]
    modulo2 = np.linalg.norm(fila2)
    suma_matriz=sum(resultado_matrices[:,l])
    resultado_coseno= (suma_matriz)/(modulo1*modulo2)
    resultado_final.append(resultado_coseno)

In [31]:
print(resultado_final)

[0.08361295236251011, 0.10029362013972987, 0.1045828318678386, 0.000900898409985926, 0.14055538360060418, 0.11969151624533524, 0.17348298231641718, 0.0832941654978965, 0.15436998364286586, 0.1508059719357134, 0.1409169782234084, 0.10923828849610065, 0.07681174905325958, 0.10996168271726219, 0.061887880492259226, 0.12352434138100016, 0.1739206779527387, 0.10306473344563839, 0.0304734826186753, 0.18636702991443255, 0.1583825358762484, 0.10328640779461758, 0.1322782604965072, 0.1322782604965072, 0.005710129373482996, 0.16709594785623322, 0.10148695089407904, 0.10658496638261672, 0.1259649239883538, 0.09350642124946748, 0.002677062401588409, 0.19187726940613067, 0.11131109176998913, 0.10892108000218784, 0.0757892875210301, 0.07587256560697549, 0.14660149768133351, 0.12786112680003106, 0.1435461647623297, 0.12334762521764472, 0.0004254892434083183, 0.15279611309857266, 0.1391237614625585, 0.1281703128674669, 0.1383894847964286, 0.15955307162325058, 0.15128470064029925, 0.05016256260301785, 

In [32]:
indice_maximo = resultado_final.index(max(resultado_final))
respuesta=None
for indice, palabra in enumerate(titulos_encontrados):
    if indice == indice_maximo:
       respuesta = palabra

In [33]:
print("El libro con mas similitud al query es: ", respuesta)

El libro con mas similitud al query es:  The_divine_comedy
